  981. Time Based Key-Value Store

Design a time-based key-value data structure that can store multiple values for the same key at different time stamps and retrieve the key's value at a certain timestamp.

Implement the TimeMap class:

TimeMap() Initializes the object of the data structure.
void set(String key, String value, int timestamp) Stores the key key with the value value at the given time timestamp.
String get(String key, int timestamp) Returns a value such that set was called previously, with timestamp_prev <= timestamp. If there are multiple such values, it returns the value associated with the largest timestamp_prev. If there are no values, it returns "".


In [ ]:
class TimeMap:

    def __init__(self):
        self.store = collections.defaultdict(list)

        

    def set(self, key: str, value: str, timestamp: int) -> None:
        self.store[key].append((timestamp, value))
        

    def get(self, key: str, timestamp: int) -> str:
        if len(self.store[key]) == 0:
            return ""      

        
        def findLastSmallerEqual(array, target): # last smaller or equal postion
            L = len(array)
            start, end = 0, L-1
            while start + 1 < end:
                mid = start + (end-start)//2
                t = array[mid][0]
                if t == target:
                    start = mid
                elif t > target:
                    end = mid-1
                elif t < target:
                    start = mid
            if array[end][0] <= target:
                return end
            if array[start][0] <= target:
                return start
            return -1
        
        array = self.store[key]
        idx = findLastSmallerEqual(array, timestamp)
        if idx == -1:
            return ""
        else:
            return array[idx][1]


        
        


# Your TimeMap object will be instantiated and called as such:
# obj = TimeMap()
# obj.set(key,value,timestamp)
# param_2 = obj.get(key,timestamp)


# bisect.bisect_left: insert position, if there are same value, insert left of it (first position bigger or equal)

# bisect.bisect_right: insert position, if there are same value, insert right of it (first position bigger than it)


In [ ]:
class TimeMap:

    def __init__(self):
        self.data = collections.defaultdict(list)

    def set(self, key: str, value: str, timestamp: int) -> None:
        self.data[key].append((timestamp, value))

    def get(self, key: str, timestamp: int) -> str:
        if key not in self.data:
            return ''
        
        idx = bisect.bisect_left(self.data[key], (timestamp + 1, ''))
        
        if idx == 0:
            return ''
        
        return self.data[key][idx - 1][1]

In [ ]:

# Implement (code) a Key value store with transactions.

# Write a Fully funcitonal code in 25-30 min in interview with test cases

# Set
# Get
# Delete are methods in Key value store

# for transactions
# Begin
# Commit
# Rollback

# Ideas are welcome,

from typing import *


class KVStore:  # nested txn 
    def __init__(self):
        self.stack = [{}]

    def set(self, key: Any, value: Any):
        """O(1)"""
        self.stack[-1][key] = value

    def get(self, key: Any) -> Optional[Any]:
        """O(transaction)"""
        for i in range(len(self.stack) - 1, -1, -1):
            if key in self.stack[i]:
                return self.stack[i][key]

    def begin(self):
        """O(1)"""
        self.stack.append({})

    def commit(self):
        """O(n_keys)"""
        last_dic = self.stack.pop()

        for k, v in last_dic.items():
            self.stack[-1][k] = v

    def rollback(self):
        """O(1)"""
        self.stack.pop()


def test_KVStore():
    kv = KVStore()
    kv.set(1, 3)

    assert kv.get(1) == 3
    assert kv.get(2) is None


def test_KVStore_single_transaction():
    kv = KVStore()
    kv.set(1, 3)

    kv.begin()
    kv.set(2, 4)
    assert kv.get(1) == 3
    assert kv.get(2) == 4
    kv.commit()

    assert kv.get(1) == 3
    assert kv.get(2) == 4


def test_KVStore_rollback():
    kv = KVStore()
    kv.set(1, 3)

    kv.begin()
    kv.set(2, 4)
    assert kv.get(1) == 3
    assert kv.get(2) == 4
    kv.rollback()

    assert kv.get(1) == 3
    assert kv.get(2) is None


def test_KVStore_multiple_begin():
    kv = KVStore()
    kv.set(1, 3)

    kv.begin()
    kv.set(2, 4)

    kv.begin()
    kv.set(3, 5)

    assert kv.get(1) == 3
    assert kv.get(2) == 4
    assert kv.get(3) == 5

    kv.commit()

    assert kv.get(1) == 3
    assert kv.get(2) == 4
    assert kv.get(3) == 5

    kv.rollback()

    assert kv.get(1) == 3
    assert kv.get(2) == None
    assert kv.get(3) == None
    
    
class Transaction:
    def __int__(self, transID):
        self.transID = transID
        self.m = {}
        self.logs = [] # WAL log

class KVStore:
  def __init__(self):
    self.m = {}
    self.transID = 0
    Self.trans = collections.defaultdict(Transaction)

  # transID: transaction ID
  def Get(self, transID, key):
    m = self.trans[transID]
    if key in m:     
      return m[key]
    else:
      self.rwLock.readLock()
      yield self.m[key]
      self.rwLock.readUnLock()

  def Set(self, transID, key, val):
    self.trans[transID].m[key] = val
    self.trans[transID].logs.append((0, key, val)

  def Del(self, transID, key, val):
    self.trans[transID].logs.append((1, key, self.trans[transID].m[key]))
    del(self.trans[transID].m, key)

  # return a transaction ID
  def Begin(self):
    transID, self.transID = self.transID, self.transID+1
    self.trans[transID] = Transaction(transID)
    Return transID

  def Commit(self, transID):
    self.rwLock.writeLock()
    ## update
    for action in self.trans[transID].logs:
      if action[0] == 0:
        Self.m[action[1]] = action[2]
      else:
        del(self.m, action[1])
    self.rwLock.writeUnLock()

  def Rollback(self, transID):
    # undo WAL logs
    del(self.trans, transID)
 


class DB:   
    
    def __init__(self):
        self.current = {}
        self.stack = []

    def begin(self):
        self.stack.append(self.current)
        self.current = dict(self.current)

    def commit(self):
        self.stack[-1] = dict(self.current)

    def end(self):
        self.current = dict(self.stack.pop())

    def set(self, key, value):
        self.current[key] = value

    def get(self, key):
        return self.current[key]

    def rollback(self):
        self.current = dict(self.stack[-1])
        
        
from collections import deque
from typing import Any, Optional


class DB:
    def __init__(self):
        self.current = {}
        self.stack = deque()
        self.open_transactions = 0

    def get(self, value: Any) -> Optional[Any]:
        return self.current.get(value)

    def set(self, key: Any, value: Any) -> None:
        if self.open_transactions == 0:
            self.current[key] = value
            return

        self.stack.append({key: value})

    def delete(self, key: int) -> None:
        self.current.pop(key)

    def commit(self):
        if self.open_transactions == 0:
            return

        changes = {}
        while len(self.stack) > 0:
            curr = self.stack.pop()

            if curr == "X_BLOCK":
                break
            # for any/all transactions add to temporary change store
            changes.update(curr)

        # finally, update main data store
        self.current.update(changes)

        # reduce transaction count
        self._update_transaction_count(-1)

    def begin(self):
        self._update_transaction_count(1)
        self.stack.append("X_BLOCK")
        self.current = dict(self.current)

    def rollback(self):
        if self.open_transactions == 0:
            return

        while len(self.stack) != 0:
            curr = self.stack.pop()
            if curr == "X_BLOCK":
                break

        self._update_transaction_count(-1)

    def _update_transaction_count(self, val: int) -> None:
        self.open_transactions += val
        self.open_transactions = abs(self.open_transactions)

In [ ]:
class TimeMap:

    def __init__(self):
        self.store = collections.defaultdict(list)

        

    def set(self, key: str, value: str, timestamp: int) -> None:
        self.store[key].append((timestamp, value))
        

    def get(self, key: str, timestamp: int) -> str:
        if len(self.store[key]) == 0:
            return ""      

        
        def findLeftInsertPos(array, target): # bisect_left: insert position
            # first bigger or equal
            L = len(array)
            start, end = 0, L-1
            while start + 1 < end:
                mid = start + (end-start)//2
                t = array[mid][0]
                if t >= target:
                    end = mid
                elif t < target:
                    start = mid+1
            if array[start][0] >= target:
                return start
            if array[end][0] >= target:
                return end
            return L
        
        array = self.store[key]
        idx = findLeftInsertPos(array, timestamp+1)
        if idx == 0:
            return ""
        else:
            return array[idx-1][1]


        
        


# Your TimeMap object will be instantiated and called as such:
# obj = TimeMap()
# obj.set(key,value,timestamp)
# param_2 = obj.get(key,timestamp)


# bisect.bisect_left: insert position, if there are same value, insert left of it (first position bigger or equal)

# bisect.bisect_right: insert position, if there are same value, insert right of it (first position bigger than it)


In [ ]:
class TimeMap:

    def __init__(self):
        self.data = collections.defaultdict(list)

    def set(self, key: str, value: str, timestamp: int) -> None:
        self.data[key].append((timestamp, value))

    def get(self, key: str, timestamp: int) -> str:
        if key not in self.data:
            return ''
        
        idx = bisect.bisect_left(self.data[key], (timestamp + 1, ''))
        
        if idx == 0:
            return ''
        
        return self.data[key][idx - 1][1]

Algorithms and Data Structures Interview: In this interview, you will design and code a data structure to accurately store time-based information.

[LC 久八一，原题没有变种，](https://leetcode.com/problems/time-based-key-value-store/description/?envType=company&envId=openai&favoriteSlug=openai-all)


自己产生一个 timestamp


1. testing case ,
following up: 
1. how to handle concurent issue,   ANS:read+write lock. 
2. server bottle neck , if server resources limited, the data uploaded in Memery, but the dataset is too big.  ANS: keep most recently hitted data section in Memory, and preload most fr‍‍‌‍‍‌‍‌‌‍‍‍‌‌‍‍‌‌‍equent datasets in Memory.
3. future time.


然后follow-up问说如果多线程同时读写该如何optimize。我给write加了个thread lock，又是现查现写。问read需不需要lock我思考了一下感觉不需要


1. KV store 里面，每个 key 存一个 sorted list，list 是一个 (timestamp, value) 的 tuple，然后 sorted by timestamp; 在 query 的时候，用一个 bisect right 来找到最大的 timestamp match to query
2. 针对多线程的 follow up，这里可能需要的就是 paging 的思想，把 key space 进行 paging 分别上锁从而能够增加并发程度
3. 针对 OOM 的 follow up，一种方法是利用 LRU 等一些方法把不太常用的 key 写到磁盘里面去节省空间。简单的方法就是直接 append 到一个 file 里面去，然后需要查找的时候就线性查找; 如果需要更加优化的方法的话，可能要考虑实现简单版本的 SST



1. Make update consistency even in multithreading。
我的理解因为多线程的情况下会产生一起update一个key with same timestamp，所以这个情况就是需要加锁保证同一时间只有一个update或者用thread safe的data structure。
2. when invoke get, pass in a future timestamp. my answer was to add a sleep in the get implementation, wait for the time to reach the input timestamp and proceed.  
我的理解是可以define system rule来处理这个情况，譬如说return null，或者说return latest value。

其实这题还有一问呢。要建 index, 我只有时间口述了下inverted index。



In [ ]:
%%file test_list.py

import time
import collections, bisect
import pytest
from datetime import datetime, timezone
class TimeMap:

    def __init__(self):
        self.data = collections.defaultdict(list)

    def set(self, key: str, value: str) -> None:

        # self.data[key].append((time.time_ns(), value))
        
        self.data[key].append((int(datetime.now(timezone.utc).timestamp()*1000000), value))
        


    def get(self, key: str, timestamp: int) -> str:
        if key not in self.data:
            return ''
        
        idx = bisect.bisect_left(self.data[key], (timestamp + 1, ''))
        
        if idx == 0:
            return ''
        
        return self.data[key][idx - 1][1]
    
def test_should_pass():
    a = 1
    b = 1
    assert a == b
pytest.main()

In [7]:
!python3 -m pytest /Users/jxhan/jason/algo/CommonAlgorithms/test_list.py 


6357.79s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


============================= test session starts ==============================
platform darwin -- Python 3.9.6, pytest-8.3.2, pluggy-1.5.0
rootdir: /Users/jxhan/jason/algo/CommonAlgorithms
collected 0 items                                                              

============================ no tests ran in 0.00s =============================
ERROR: file or directory not found: /Users/jxhan/jason/algo/CommonAlgorithms/test_list.py



In [10]:
import time
from datetime import datetime, timezone
print(time.time_ns()+1)
now = datetime.now()
timestamp_str = now.strftime("%Y-%m-%d %H:%M:%S.%f")
print(timestamp_str)
timestamp_str = now.strftime("%Y-%m-%d-%H-%M-%S-%f")
print(timestamp_str)
utc_now = datetime.utcnow()
timestamp_str = utc_now.strftime("%Y-%m-%d-%H-%M-%S-%f")
print(timestamp_str)


# Get the current UTC time
utc_now = datetime.now(timezone.utc)

# Get the current UTC time in microseconds
microseconds = utc_now.microsecond
print(int(utc_now.timestamp()*1000000))

1725420201355977001
2024-09-03 20:23:21.356037
2024-09-03-20-23-21-356037
2024-09-04-03-23-21-356170
1725420201356229


In [16]:
import glob
import os

def find_files_by_pattern(directory, pattern):
    return [f.replace(directory, '').replace('.ipynb', '') for f in glob.glob(os.path.join(directory, pattern))]

files = find_files_by_pattern('./', '*.*')
print(files)

['Graph Algorithms', 'A', 'Basic Algorithms', 'Door', 'Two Pointers', 'Pin', 'Binary Search Tree', 'CP', 'hacker-rank', 'LinkedList', 'System design', 'oracle', 'BFS', 'Ms', 'FB', 'Coin', 'SelfTest', 'Trie', 'phone_numbers.txt', 'top-100-liked', 'OAI', 'Greedy', 'ByteDance', 'Hash Table', 'DFS', 'Backtracking', 'LT-75', 'Union Find', 'Effective and Bug-free coding principles', 'top-150', 'Dynamic Programming', 'YaTi', 'Strings', 'PremiumAlgo100LeetCode', 'Advanced Language Features', 'Binary Tree Divide and conquer', 'Problem Types', 'Rob', 'Binary Search', 'Bit manipulation', 'Stack and Queue', 'Segment Tree', 'Song', 'Priority Queue']
